In [2]:
# Google colab
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Kaggle_competition")
#os.listdir()

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd

In [5]:
import sys 
sys.path.append("src")
import src.support as sp

In [33]:
#df = pd.read_csv("data/train.csv")
df = pd.read_csv("data/train.csv", index_col=0)
df.head(2)

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183


## 0. Gestión de outliers

Tras el análisis realizado en el EDA se puede apreciar que la columna carat explica de manera significativa el precio, y crece hasta llegar al valor 1.96. A partir de ahí encontramos los precios más elevados y aumenta la importancia del las variables color, clarity y cut para explicarlos. Por este motivo la gestión de outliers consiste en dividir el df en ese punto y realizar el entrenamiento por separado.

In [34]:
bins = [0,1.96,10]  
labels = ['df_1', 'df_2']  
df['part_df'] = pd.cut(df['carat'], bins=bins, labels=labels)

df_1 = df[df['part_df'] == 'df_1']
df_2 = df[df['part_df'] == 'df_2']
df_1 = df_1.drop(['x', 'y', 'z','depth','part_df'], axis=1)
df_2 = df_2.drop(['x', 'y', 'z','depth','part_df'], axis=1)
df_1= pd.DataFrame(df_1).reset_index()
df_2= pd.DataFrame(df_2).reset_index()


In [35]:
df = df.drop(['x', 'y', 'z','depth'], axis=1)

In [36]:
df.select_dtypes(include = np.number).describe().T

,count,mean,std,min,25%,50%,75%,max
carat,40455.0,0.797576,0.475939,0.200,0.400,0.70,1.040,5.010
table,40455.0,57.451694,2.234569,44.000,56.000,57.00,59.000,95.000
price,40455.0,7.783822,1.016828,5.787,6.851,7.78,8.581,9.842


In [ ]:
df_1.select_dtypes(include = np.number).describe().T

,count,mean,std,min,25%,50%,75%,max
id,38793.0,20229.281365,11674.836794,0.000,10130.000,20229.000,30349.000,40454.000
carat,38793.0,0.740269,0.392497,0.200,0.390,0.700,1.020,1.960
price,38793.0,7.706724,0.965140,5.787,6.825,7.716,8.494,9.842


In [ ]:
df_2.select_dtypes(include = np.number).describe().T

,count,mean,std,min,25%,50%,75%,max
id,1662.0,20173.750301,11767.017335,40.000,9747.250,20108.000,30167.250,40424.000
carat,1662.0,2.135187,0.228131,1.970,2.010,2.040,2.180,5.010
price,1662.0,9.583392,0.218849,8.527,9.478,9.634,9.743,9.842


## 1. Encoding

Todas las variables no numéricas tienen orden, por lo que utilizaremos ordinal encoder sobre las columnas color, clarity y cut.

In [37]:
# Encoding df_1

orden_color = ['J', 'I', 'H', 'G', 'F', 'D', 'E']
orden_clarity = ['SI2', 'I1', 'SI1', 'VS2', 'VS1', 'VVS2', 'IF', 'VVS1']
orden_cut = ['Fair', 'Premium', 'Good', 'Very Good', 'Ideal']

df_1 = sp.Preprocesado(df_1).ordinal_encoder( "color", orden_color)
df_1 = sp.Preprocesado(df_1).ordinal_encoder( "clarity", orden_clarity)
df_1 = sp.Preprocesado(df_1).ordinal_encoder( "cut", orden_cut)

df_1 = df_1.drop(['color', 'clarity', 'cut'], axis=1)

In [38]:
# Encoding df_2

orden_color = ['D', 'E', 'F', 'I', 'H', 'G', 'J']
orden_clarity = ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']
orden_cut = ['Ideal','Very Good', 'Premium', 'Good','Fair']

df_2 = sp.Preprocesado(df_2).ordinal_encoder( "color", orden_color)
df_2 = sp.Preprocesado(df_2).ordinal_encoder( "clarity", orden_clarity)
df_2 = sp.Preprocesado(df_2).ordinal_encoder( "cut", orden_cut)

df_2 = df_2.drop(['color', 'clarity', 'cut'], axis=1)

In [26]:
df_2.head(2)

,id,carat,table,price,color_oe,clarity_oe,cut_oe
0,40,2.53,57.0,9.626,4,6,2
1,85,2.09,64.0,9.357,6,4,3


In [27]:
df_2.head(2)

,id,carat,table,price,color_oe,clarity_oe,cut_oe
0,40,2.53,57.0,9.626,4,6,2
1,85,2.09,64.0,9.357,6,4,3


## 2. Estandarización

In [40]:
df_1= sp.Preprocesado(df_1).robust_scaler(['carat', 'table'])
df_2= sp.Preprocesado(df_2).robust_scaler(['carat', 'table'])

In [39]:
df_2.head(2)

,id,carat,table,price,color_oe,clarity_oe,cut_oe
0,40,2.53,57.0,9.626,4,6,2
1,85,2.09,64.0,9.357,6,4,3


## 3. Predicción

#### 3.1. Decision Tree

In [ ]:
resultados_dt_1= sp.Prediccion(df_1).decision_tree_training('price', 250, 250)

In [ ]:
resultados_dt_2= sp.Prediccion(df_2).decision_tree_training('price', 250, 250)

In [ ]:
resultados_dt_1

In [ ]:
resultados_dt_2

#### 3.2. Random Forest

In [ ]:
resultados_rf_1 = sp.Prediccion(df_1).random_forest_training('price', 250, 250)

In [ ]:
resultados_rf_2 = sp.Prediccion(df_2).random_forest_training('price', 250, 250)

In [ ]:
resultados_rf_1

In [ ]:
resultados_rf_2

#### 3.3. Gradient Boosting

In [ ]:
resultados_gb_1 = sp.Prediccion(df_1).gradient_training('price', 250, 250)

Fitting 10 folds for each of 5184 candidates, totalling 51840 fits


In [ ]:
resultados_gb_2 = sp.Prediccion(df_2).gradient_training('price', 250, 250)

In [ ]:
resultados_gb_1

In [ ]:
resultados_gb_2

## SUBMISION

In [ ]:
df_test = pd.read_csv("data/test.csv", index_col=0)

In [ ]:
bins = [0,1.96,10]  
labels = ['df_1', 'df_2']  
df_test['part_df'] = pd.cut(df_test['carat'], bins=bins, labels=labels)

df_1_t = df_test[df_test['part_df'] == 'df_1']
df_2_t = df_test[df_test['part_df'] == 'df_2']
df_1_t = df_1_t.drop(['x', 'y', 'z','table','depth','part_df'], axis=1)
df_2_t = df_2_t.drop(['x', 'y', 'z','table','depth','part_df'], axis=1)
df_1_t= pd.DataFrame(df_1_t).reset_index()
df_2_t= pd.DataFrame(df_2_t).reset_index()

In [ ]:
# Encoding df_1

orden_color = ['J', 'I', 'H', 'G', 'F', 'D', 'E']
orden_clarity = ['SI2', 'I1', 'SI1', 'VS2', 'VS1', 'VVS2', 'IF', 'VVS1']
orden_cut = ['Fair', 'Premium', 'Good', 'Very Good', 'Ideal']

df_1_t = sp.Preprocesado(df_1_t).ordinal_encoder( "color", orden_color)
df_1_t = sp.Preprocesado(df_1_t).ordinal_encoder( "clarity", orden_clarity)
df_1_t = sp.Preprocesado(df_1_t).ordinal_encoder( "cut", orden_cut)

df_1_t = df_1_t.drop(['color', 'clarity', 'cut'], axis=1)

In [ ]:
# Encoding df_2

orden_color = ['D', 'E', 'F', 'I', 'H', 'G', 'J']
orden_clarity = ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']
orden_cut = ['Ideal','Very Good', 'Premium', 'Good','Fair']

df_2_t = sp.Preprocesado(df_2_t).ordinal_encoder( "color", orden_color)
df_2_t = sp.Preprocesado(df_2_t).ordinal_encoder( "clarity", orden_clarity)
df_2_t = sp.Preprocesado(df_2_t).ordinal_encoder( "cut", orden_cut)

df_2_t = df_2_t.drop(['color', 'clarity', 'cut'], axis=1)

In [ ]:
prediccion_tree_1= sp.Prediccion(df_1).decision_tree_predictor('price',20, 2, 50, 50)
prediccion_tree_2= sp.Prediccion(df_2).decision_tree_predictor('price',20, 2, 50, 50)
df_1["price"] = prediccion_tree_1.predict(df_1)
df_2["price"] = prediccion_tree_2.predict(df_2)

In [ ]:
prediccion_gradient_1= sp.Prediccion(df_1).gradient_predictor('price',16, 2, 150, 75)
prediccion_gradient_2= sp.Prediccion(df_2).gradient_predictor('price',12, 2, 50, 25)

In [ ]:
df_1_t["price"] = prediccion_gradient_1.predict(df_1_t)
df_2_t["price"] = prediccion_gradient_2.predict(df_2_t)

In [ ]:
df_concat = pd.concat([df_1_t, df_2_t])
df_concat = df_concat.set_index('id')

In [ ]:
df_concat = df_concat.sort_values('id', ascending=True)
df_concat= df_concat.drop(['color_oe', 'clarity_oe', 'cut_oe', 'carat'], axis=1)


,carat,color_oe,clarity_oe,cut_oe,price
id,,,,,
0,0.32,1,2,4,6.202274
1,1.24,1,2,1,8.625183
2,1.66,5,2,1,9.504222
3,0.75,5,0,1,7.915786
4,1.50,6,0,0,8.921921
...,...,...,...,...,...
13480,1.10,3,2,1,8.529770
13481,0.90,5,2,3,8.378268
13482,0.30,4,3,4,6.488996


In [ ]:
df_concat=df_concat.rename(columns={'index': 'id'})
df_concat.to_csv("data/submission_gradient_1.csv")